## Create the path of the player 
Preprocessing for the world map visualisation. The goal is to extract the successive location and date of every match played by every players. At the end we want a list for each player, in chronological order of the time and cities of every match they played.

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import time
import datetime

In [26]:
df = pd.read_csv("data/tennis_data_cleaned.csv")

In [27]:
df.columns

Index(['Unnamed: 0', 'ATP', 'Location', 'Tournament', 'Date', 'Series',
       'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank',
       'LRank', 'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4',
       'W5', 'L5', 'Wsets', 'Lsets', 'Comment', 'B365W', 'B365L', 'pl1_flag',
       'pl1_year_pro', 'pl1_weight', 'pl1_height', 'pl1_hand', 'pl2_flag',
       'pl2_year_pro', 'pl2_weight', 'pl2_height', 'pl2_hand'],
      dtype='object')

In [3]:
player_names = set()

In [4]:
for name in df.Winner.unique():
    player_names.add(name)

for name in df.Loser.unique():
    player_names.add(name)
player_names = list(player_names)

In [5]:
player_path = {}

In [18]:
s = "2030-01-01"
time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple())

1893452400.0

In [39]:
mn = 1e40
mx = 0

mx_rank = 0
player_path = {}
for name in player_names:

    path_L = df[df.Loser == name][['Location', 'Date', 'LRank']].values.tolist()
    path_W = df[df.Winner == name][['Location', 'Date', 'WRank']].values.tolist()
    path = path_L + path_W
    path.insert(0, ["END", "2030-01-01", "-1"])
    path.append(["START", "2000-01-01", "-1" ])
    for i,a in enumerate(path):
        t = time.mktime(datetime.datetime.strptime(a[1], "%Y-%m-%d").timetuple())
        path[i].insert(2,t)
        mn = min(t, mn)
        mx = max(t, mx)
   
        mx_rank = max(mx_rank, int(path[i][3]))
        
    path.sort(key=lambda x: x[2])
    
    player_path[name] = path.copy()
print(mx_rank)

1890


In [34]:
player_path[name]

[['START', '2000-01-01', 946681200.0, '-1'],
 ['Estoril', '2008-04-14', 1208124000.0, 146.0],
 ['Estoril', '2008-04-18', 1208469600.0, 146.0],
 ['Estoril', '2008-04-18', 1208469600.0, 146.0],
 ['Paris', '2008-05-25', 1211666400.0, 134.0],
 ['London', '2008-06-24', 1214258400.0, 110.0],
 ['New York', '2008-08-26', 1219701600.0, 90.0],
 ['Johannesburg', '2009-02-03', 1233615600.0, 99.0],
 ['Johannesburg', '2009-02-05', 1233788400.0, 99.0],
 ['Johannesburg', '2009-02-06', 1233874800.0, 99.0],
 ['Johannesburg', '2009-02-07', 1233961200.0, 99.0],
 ['Costa Do Sauipe', '2009-02-10', 1234220400.0, 86.0],
 ['Costa Do Sauipe', '2009-02-11', 1234306800.0, 86.0],
 ['Costa Do Sauipe', '2009-02-13', 1234479600.0, 86.0],
 ['Costa Do Sauipe', '2009-02-13', 1234479600.0, 86.0],
 ['Miami', '2009-03-26', 1238022000.0, 74.0],
 ['Miami', '2009-03-28', 1238194800.0, 74.0],
 ['Miami', '2009-03-30', 1238364000.0, 74.0],
 ['Casablanca', '2009-04-06', 1238968800.0, 70.0],
 ['Casablanca', '2009-04-08', 123914160

### Write to a file

In [35]:
with open('data/player_trajectories.json', 'w') as outfile:
    outfile.write(json.dumps(player_path))